In [1]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils.io_utils import HDF5Matrix

# data path
path_csi =  'J:\\Data\\Wi-Fi_processed\\'
path_csi_hc = 'J:\\Data\\Wi-Fi_HC\\180_100\\'
path_csi_hdfs = 'J:\\Data\\Wi-Fi_Processed_HDFS\\'

# data info
df_info = pd.read_csv('data_subc_sig_v1.csv')
#df_info = df_info[df_info.id_person < 50]

person_uid = np.unique(df_info['id_person'])
dict_id = dict(zip(person_uid,np.arange(len(person_uid))))

C:\Users\herok\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Parameters
max_value = np.max(df_info['max'].values)
no_classes = len(dict_id)
num_csi = 15000
csi_time = 500 #data_csi.shape[1]
csi_subc = 30
input_shape = (csi_time,csi_subc,6)
filename = 'csi_1_1.hdf5'

In [3]:
f1 = h5py.File(path_csi_hdfs+'csi_1_1.hdf5','r')
f1['csi'].shape

(891, 15000, 30, 2, 3)

In [4]:
# nomalization function
def prep_data(array):
    arr_abs = np.abs(array)[:,:csi_time,:,:,:] / max_value
    return arr_abs.reshape([-1,csi_time,csi_subc,6])
def prep_label(array):
    arr_label = array[:,0]
    # Label
    label = np.array([dict_id[arr] for arr in arr_label])
    return tf.keras.utils.to_categorical(label, no_classes)

In [5]:
# Train Test Split
x_train = HDF5Matrix(path_csi_hdfs+filename,'csi',start=0,end=800,normalizer=prep_data)
x_test = HDF5Matrix(path_csi_hdfs+filename,'csi',start=800,end=900,normalizer=prep_data)
y_train = HDF5Matrix(path_csi_hdfs+filename,'label',start=0,end=800,normalizer=prep_label)
y_test = HDF5Matrix(path_csi_hdfs+filename,'label',start=800,end=900,normalizer=prep_label)

In [6]:
#models
def simple_cnn(input_shape):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        activation='relu',
        input_shape=input_shape
    ))
    model.add(tf.keras.layers.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        activation='relu'
    ))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(rate=0.3))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=256, activation='relu'))
    model.add(tf.keras.layers.Dropout(rate=0.3))
    model.add(tf.keras.layers.Dense(units=no_classes, activation='softmax'))
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
                  optimizer=tf.keras.optimizers.Adam(lr=0.1),
                  metrics=['accuracy'])
    return model

In [ ]:
simple_cnn_model = simple_cnn(input_shape)
simple_cnn_model.fit(x_train, y_train, epochs=10,batch_size=10,shuffle='batch')

Epoch 1/10
800/800 [==============================] - 250s 313ms/step - loss: 15.9337 - acc: 0.0025
Epoch 2/10
800/800 [==============================] - 250s 312ms/step - loss: 15.8763 - acc: 0.0150
Epoch 3/10
800/800 [==============================] - 244s 306ms/step - loss: 15.8763 - acc: 0.0150
Epoch 4/10
800/800 [==============================] - 247s 309ms/step - loss: 15.8763 - acc: 0.0150
Epoch 5/10
800/800 [==============================] - 253s 317ms/step - loss: 15.8763 - acc: 0.0150
Epoch 6/10
720/800 [==========================>...] - ETA: 24s - loss: 15.8942 - acc: 0.0139

In [ ]:

# Plot
def plot_loss(history):
    plt.plot(history.history['loss'])
    #plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train','Test'],loc=0)
def plot_acc(history):
    plt.plot(history.history['acc'])
    #plt.plot(history.history['val_acc'])
    plt.title('Model Acc')
    plt.ylabel('Acc')
    plt.xlabel('Epoch')
    plt.legend(['Train','Test'],loc=0)

plot_loss(simple_cnn_model.history)
plot_acc(simple_cnn_model.history)


In [ ]:
simple_cnn_model.history.history['acc']